In [1]:
#Fix quarterly call schedules for caregiver given the birth/enrollment visit

In [ ]:
# Imports
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

In [4]:
mismtch = []
def fix_schedule(mv):
#     breakpoint()
    mismtch = []
    try:
        if 'sec' in mv.schedule_name:
            x_name = 'cohort_' + mv.schedule_name[:5]
            vs = mv.schedule_name.split('_')
            fschedule_name = '_'.join([vs[0], 'sec',  vs[1].replace('sec', 'quart'), vs[2]])
        elif 'birth' in mv.schedule_name:
            x_name = 'cohort_' + mv.schedule_name[:1]
            vs = mv.schedule_name.split('_')
            fschedule_name = '_'.join([vs[0], vs[1].replace('birth', 'quarterly'), vs[2]])   
        else:
            x_name = 'cohort_' + mv.schedule_name[:1]
            vs = mv.schedule_name.split('_')
            fschedule_name = '_'.join([vs[0], vs[1].replace('enrol', 'quarterly'), vs[2]])
        
        next_appt = Appointment.objects.get(subject_identifier=mv.subject_identifier,
                                            visit_code='2001M', visit_code_sequence=0,
                                           schedule_name=fschedule_name)
    except Appointment.DoesNotExist:
        print(mv.subject_identifier)
        print("hmmmmmm!")
    else:
        onschedule_cls =  django_apps.get_model(next_appt.schedule.onschedule_model)
        
        try:
            ons = onschedule_cls.objects.get(subject_identifier=mv.subject_identifier,
                                            schedule_name__startswith=fschedule_name)
        except onschedule_cls.DoesNotExist:
            print('whoa!')
        else:
            print(mv.report_datetime.date(), ons, ons.onschedule_datetime.date())
            if mv.report_datetime.date() != ons.onschedule_datetime.date():
                print(">>>>>>", mv.subject_identifier, mv.report_datetime.date(), ons.onschedule_datetime.date())
                mismtch.append(mv.subject_identifier)
                if 'x' in ons.schedule_name:
                    schedule_name = ons.schedule_name[:-1]
                else:
                    schedule_name = ons.schedule_name

                ssh = SubjectScheduleHistory.objects.get(subject_identifier=ons.subject_identifier,
                                                   schedule_name=ons.schedule_name)

                ons.schedule_name = schedule_name + 'x'
                ons.save()

                ap0 = Appointment.objects.filter(subject_identifier=mv.subject_identifier,
                                  schedule_name=ons.schedule_name,
                                  visit_code_sequence=0)

                for ap in ap0:
                    ap0.schedule_name = ons.schedule_name
                    ap0.save()


                ssh.delete()
                mv.save()

                for ip in  ap0.exclude(appt_status='New'):
                    ap1 = Appointment.objects.get(subject_identifier=ip.subject_identifier,
                                                   visit_code=ip.visit_code,
                                                   schedule_name=schedule_name,
                                                   visit_code_sequence=0)


                    try:
                        v1 = MaternalVisit.objects.get(appointment=ip)
                        v1.appointment = ap1
                        v1.save()
                    except:
                        pass

                    ap1.user_modified = ap0.user_modified
                    ap1.appt_status = ap0.appt_status
                    ap1.appt_reason = ap0.appt_reason
                    ap1.appt_datetime = ap0.appt_datetime

                    ap1.save()


                ap0.delete()
                ons.delete()


In [7]:
# mv = MaternalVisit.objects.get(visit_code='2000D', subject_identifier='B142-040990572-5')
# fix_schedule(mv)

mv = MaternalVisit.objects.get(visit_code='2000D', subject_identifier='B142-040990575-8')
fix_schedule(mv)

2022-01-25 B142-040990575-8 2022-01-25 2022-01-25


In [3]:
for mv in MaternalVisit.objects.filter(visit_code__startswith='2000', visit_code_sequence=0):
    fix_schedule(mv)

B142-040990001-5 2000M
>>>>>> B142-040990001-5 2021-05-20 2021-06-03
B142-040990002-3 2000M
>>>>>> B142-040990002-3 2021-05-07 2021-05-28
B142-040990003-1 2000M
>>>>>> B142-040990003-1 2021-05-07 2021-05-28
B142-040990005-6 2000M
>>>>>> B142-040990005-6 2021-05-10 2021-06-04
B142-040990006-4 2000M
>>>>>> B142-040990006-4 2021-06-04 2021-05-06
B142-040990007-2 2000M
B142-040990008-0 2000M
>>>>>> B142-040990008-0 2021-05-14 2021-05-07
B142-040990009-8 2000M
>>>>>> B142-040990009-8 2021-05-12 2021-05-07
B142-040990010-6 2000M
>>>>>> B142-040990010-6 2021-05-12 2021-05-10
B142-040990011-4 2000M
>>>>>> B142-040990011-4 2021-05-17 2021-05-10
B142-040990013-0 2000M
>>>>>> B142-040990013-0 2021-05-18 2021-05-31
B142-040990014-8 2000M
>>>>>> B142-040990014-8 2021-05-24 2021-05-12
B142-040990015-5 2000M
>>>>>> B142-040990015-5 2021-05-21 2021-06-02
B142-040990016-3 2000M
>>>>>> B142-040990016-3 2021-05-19 2021-06-04
B142-040990017-1 2000M
>>>>>> B142-040990017-1 2021-05-21 2021-05-14
B142-040990

B142-040990194-8 2000M
>>>>>> B142-040990194-8 2021-10-05 2021-07-13
B142-040990195-5 2000M
>>>>>> B142-040990195-5 2021-09-16 2021-07-13
B142-040990197-1 2000M
B142-040990198-9 2000M
>>>>>> B142-040990198-9 2021-09-17 2021-07-13
B142-040990199-7 2000M
B142-040990200-3 2000M
>>>>>> B142-040990200-3 2021-09-21 2021-07-13
B142-040990201-1 2000M
>>>>>> B142-040990201-1 2021-09-23 2021-07-14
B142-040990202-9 2000M
B142-040990203-7 2000M
>>>>>> B142-040990203-7 2021-09-20 2021-07-14
B142-040990204-5 2000M
B142-040990205-2 2000M
>>>>>> B142-040990205-2 2021-09-24 2021-07-14
B142-040990206-0 2000M
>>>>>> B142-040990206-0 2021-10-07 2021-07-14
B142-040990207-8 2000M
B142-040990208-6 2000M
B142-040990209-4 2000M
B142-040990210-2 2000M
>>>>>> B142-040990210-2 2021-09-24 2021-07-15
B142-040990212-8 2000M
>>>>>> B142-040990212-8 2021-09-28 2021-07-15
B142-040990213-6 2000M
B142-040990214-4 2000M
B142-040990215-1 2000M
>>>>>> B142-040990215-1 2021-09-29 2021-07-15
B142-040990216-9 2000M
B142-040990

B142-040990440-5 2000D
>>>>>> B142-040990440-5 2022-01-25 2022-03-15
B142-040990441-3 2000M
B142-040990442-1 2000M
B142-040990443-9 2000D
>>>>>> B142-040990443-9 2022-01-31 2022-03-07
B142-040990444-7 2000M
B142-040990445-4 2000D
>>>>>> B142-040990445-4 2022-02-21 2022-02-22
B142-040990446-2 2000D
B142-040990447-0 2000M
B142-040990448-8 2000M
B142-040990449-6 2000M
B142-040990450-4 2000M
B142-040990451-2 2000M
B142-040990452-0 2000M
B142-040990453-8 2000M
B142-040990454-6 2000D
B142-040990455-3 2000M
B142-040990456-1 2000M
B142-040990457-9 2000M
B142-040990458-7 2000M
B142-040990459-5 2000M
B142-040990460-3 2000M
B142-040990461-1 2000M
B142-040990463-7 2000M
B142-040990464-5 2000M
B142-040990465-2 2000M
B142-040990466-0 2000M
B142-040990467-8 2000M
B142-040990468-6 2000M
B142-040990469-4 2000M
B142-040990470-2 2000M
B142-040990471-0 2000M
B142-040990472-8 2000M
B142-040990473-6 2000M
B142-040990474-4 2000D
B142-040990475-1 2000M
B142-040990476-9 2000M
B142-040990477-7 2000M
B142-040990

B142-040990860-4 2000M
B142-040990861-2 2000M
B142-040990862-0 2000M
B142-040990863-8 2000M
B142-040990864-6 2000M
B142-040990865-3 2000M
B142-040990866-1 2000M
B142-040990868-7 2000M
B142-040990869-5 2000M
B142-040990871-1 2000M
B142-040990872-9 2000D
B142-040990873-7 2000M
B142-040990875-2 2000M
B142-040990877-8 2000M
B142-040990878-6 2000M
B142-040990879-4 2000M
B142-040990880-2 2000M
B142-040990882-8 2000M
B142-040990883-6 2000M
B142-040990884-4 2000M
B142-040990885-1 2000M
B142-040990890-1 2000M
B142-040990891-9 2000M
B142-040990892-7 2000M
B142-040990894-3 2000M
B142-040990896-8 2000M
B142-040990898-4 2000M
B142-040990899-2 2000M
B142-040990900-8 2000M
B142-040990901-6 2000M
B142-040990902-4 2000M
B142-040990904-0 2000M
B142-040990905-7 2000M
B142-040990906-5 2000M
B142-040990907-3 2000M
B142-040990910-7 2000M
B142-040990911-5 2000M
B142-040990913-1 2000M
B142-040990918-0 2000M
B142-040990923-0 2000M
B142-040990927-1 2000M
B142-040990930-5 2000M
B142-040990932-1 2000M
B142-040990

In [3]:
ons = OnScheduleChildCohortAQuarterly.objects.get(subject_identifier='B142-040990672-3-10')

ons.onschedule_datetime

Appointment.objects.filter(subject_identifier='B142-040990672-3-10', schedule_name=ons.schedule_name).delete()

ons.delete()

sv = ChildVisit.objects.get(subject_identifier='B142-040990672-3-10', visit_code='2000D', visit_code_sequence=0)

sv.save()